# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [54]:
tf.__version__

'2.5.0'

### Importar o dataset

In [55]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [56]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [57]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [58]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [60]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [61]:
np.shape(X_train)

(8000, 12)

In [62]:
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [63]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [64]:
ann.add(tf.keras.layers.Dense(units=12, activation='tanh'))

### Adding the second hidden layer

In [65]:
ann.add(tf.keras.layers.Dense(units=12, activation='tanh'))

### Adding the output layer

In [66]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [67]:
ann.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [68]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
250/250 [==============================] - 1s 1ms/step - loss: 0.5590 - accuracy: 0.7246
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.7984
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4435 - accuracy: 0.8043
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4358 - accuracy: 0.8084
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4315 - accuracy: 0.8065
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4283 - accuracy: 0.8074
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4259 - accuracy: 0.8071
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4238 - accuracy: 0.8080
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4219 - accuracy: 0.8091
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4198 - accuracy: 0.8099

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [69]:
y_pred = ann.predict(X_test)
y_pred

array([[0.27989447],
       [0.2448374 ],
       [0.1762082 ],
       ...,
       [0.197988  ],
       [0.12772638],
       [0.22142953]], dtype=float32)

In [70]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [71]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1526   69]
 [ 207  198]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)